In [4]:
#install.packages("tidyverse")
#install.packages("fixest")
library(tidyverse)
library(fixest)


In [5]:
data <- read_csv('https://osf.io/5c3rf/download')
View(data)


Rows: 249 Columns: 27
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (26): personid, treatment, type, quitjob, age, costofcommute, children, ...
lgl  (1): ordertaker

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


personid,treatment,ordertaker,type,quitjob,age,costofcommute,children,male,married,⋯,ageyoungestchild,rental,bedroom,second_technical,high_school,tertiary_technical,university,internet,phonecalls0,phonecalls1
<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3906,1,FALSE,2,0,33,8,1,0,1,⋯,10,0,1,0,1,0,0,1,0.000,0.000
4122,1,TRUE,1,0,30,18,0,0,0,⋯,0,0,0,0,1,0,0,1,21.335,21.797
4448,0,FALSE,2,0,35,10,1,0,1,⋯,11,0,1,0,1,0,0,1,0.000,0.000
4942,1,FALSE,4,0,27,20,0,0,1,⋯,0,0,1,1,0,0,0,1,0.000,0.000
5018,1,FALSE,4,0,29,15,1,0,1,⋯,0,0,1,1,0,0,0,1,0.000,0.000
5614,1,FALSE,5,0,32,0,0,1,1,⋯,0,0,1,0,1,0,0,1,0.000,0.000
6278,0,TRUE,1,1,32,12,1,0,1,⋯,5,0,1,0,0,1,0,1,11.701,8.729
6362,0,FALSE,4,0,29,12,1,0,1,⋯,3,0,1,1,0,0,0,1,0.000,0.000
6364,0,FALSE,4,0,26,12,1,0,1,⋯,2,0,1,1,0,0,0,1,0.000,0.000


In [6]:
# Number of treated and control units
table(data$treatment)
table(data$ordertaker)
table(data$treatment, data$ordertaker)


  0   1 
118 131 


FALSE  TRUE 
  115   134 

   
    FALSE TRUE
  0    52   66
  1    63   68

In [7]:
# It is a prepared, cleaned dataset, one thing is needed: modifying the ageyoungestchild variable
table(data$ageyoungestchild)
data$ageyoungestchild <- ifelse(data$children == 0, NA, data$ageyoungestchild)


  0   1   2   3   4   5   6   7   8  10  11  13 
210  10   7   8   3   2   1   2   1   2   2   1 

In [8]:
data_temp <- data %>% 
  select(ordertaker, age:perform10, prior_experience:internet)


In [9]:
# Generating empty table-elements
vars <- colnames(data_temp)
rm(data_temp)
mean_t <- c()
mean_c <- c()
sd <- c()
p_value <- c()
model <- c()

# Generating statistics to the table
for(i in vars){
  model <- lm(paste(i, "~treatment"), data=data) 
  mean_c[i] <- mean(subset(data, treatment == 0)[[paste(i)]], na.rm=T)
  mean_t[i] <- mean(subset(data, treatment == 1)[[paste(i)]], na.rm=T)
  p_value[i] <- anova(model)$'Pr(>F)'[1]
  sd[i] <- sd(data[[paste(i)]], na.rm=T)
}

# Filling table
table <- data.frame(round(mean_t, 2), round(mean_c, 2), round(sd, 2), round(p_value, 2))
col.names <- c("Treatment mean", "Control mean", "Std.dev.", "p-value of test of equal means")  
names(table) <- col.names
print(table)

                   Treatment mean Control mean Std.dev.
ordertaker                   0.52         0.56     0.50
age                         24.44        24.35     3.55
costofcommute                7.89         8.34     6.96
children                     0.11         0.24     0.38
male                         0.47         0.47     0.50
married                      0.22         0.32     0.44
perform10                   -0.03        -0.04     0.58
prior_experience            18.96        16.75    25.88
tenure                      26.14        28.25    21.92
basewage                  1539.86      1562.80   161.45
bonus                     1030.90      1092.59   625.33
grosswage                 2949.73      3003.36   789.63
ageyoungestchild             4.60         3.00     3.35
rental                       0.24         0.20     0.42
bedroom                      0.97         0.99     0.14
second_technical             0.46         0.47     0.50
high_school                  0.18         0.14  

In [10]:
########## Regression
# Regression 1: ATE estimates, no covariates
reg1 <- feols(quitjob ~ treatment, data=data , vcov = "HC1")
reg2 <- feols(phonecalls1 ~ treatment, data=data[data$ordertaker==1, ], vcov = "HC1")
etable(reg1,reg2,fitstat = c('n','r2'))

,,reg1,reg2
,<chr>,<chr>,<chr>
1,Dependent Var.:,quitjob,phonecalls1
2,,,
3,Constant,0.3475*** (0.0440),10.06*** (0.7504)
4,treatment,-0.1872*** (0.0545),4.039*** (0.9891)
5,_______________,___________________,_________________
6,S.E. type,Heteroskedast.-rob.,Heteroskeda.-rob.
7,Observations,249,134
8,R2,0.04670,0.11258


In [11]:
# Regression 2: ATE estimates, with covariates
reg3 <- feols(quitjob ~ treatment + married + children, data=data, vcov = "HC1")
reg4 <- feols(phonecalls1 ~ treatment + married + children, data=data[data$ordertaker==1, ], vcov = "HC1")
etable(reg3,reg4,fitstat = c('n','r2'))

,,reg3,reg4
,<chr>,<chr>,<chr>
1,Dependent Var.:,quitjob,phonecalls1
2,,,
3,Constant,0.3628*** (0.0493),10.65*** (0.7589)
4,treatment,-0.1866*** (0.0556),4.056*** (0.9562)
5,married,-0.1290. (0.0742),-5.439* (2.170)
6,children,0.1103 (0.0972),3.872 (2.408)
7,_______________,___________________,_________________
8,S.E. type,Heteroskedast.-rob.,Heteroskeda.-rob.
9,Observations,249,134


In [12]:
smallsample <- filter(data, ordertaker==1 & quitjob==0)
reg5 <- feols(phonecalls1 ~ treatment, data=smallsample, vcov = "HC1")
reg6 <- feols(phonecalls1 ~ treatment + married + children, data=smallsample, vcov = "HC1")
etable(reg5,reg6,fitstat = c('n','r2'))


,,reg5,reg6
,<chr>,<chr>,<chr>
1,Dependent Var.:,phonecalls1,phonecalls1
2,,,
3,Constant,13.86*** (0.6578),14.17*** (0.5678)
4,treatment,1.747* (0.8430),1.869* (0.7856)
5,married,,-5.907. (3.002)
6,children,,5.770. (3.191)
7,_______________,_________________,_________________
8,S.E. type,Heteroskeda.-rob.,Heteroskeda.-rob.
9,Observations,95,95


In [13]:
# Phonecalls regression together
etable(reg2,reg4,reg6,fitstat = c('n','r2'))

,,reg2,reg4,reg6
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,phonecalls1,phonecalls1,phonecalls1
2,,,,
3,Constant,10.06*** (0.7504),10.65*** (0.7589),14.17*** (0.5678)
4,treatment,4.039*** (0.9891),4.056*** (0.9562),1.869* (0.7856)
5,married,,-5.439* (2.170),-5.907. (3.002)
6,children,,3.872 (2.408),5.770. (3.191)
7,_______________,_________________,_________________,_________________
8,S.E. type,Heteroskeda.-rob.,Heteroskeda.-rob.,Heteroskeda.-rob.
9,Observations,134,134,95
